In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, inspect
from src.data_clean import DataClean
from src.utils import padroniza_str, create_db

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv",
    index_col=0)

In [ ]:
configs = pd.read_excel("assets/metadado.xlsx", sheet_name="schema")
configs.head(2)

### Tratamento da base de dados

In [ ]:
df["date_time"] =  pd.to_datetime(df[["year", "month", "day", "hour", "minute"]],  dayfirst=True)
df.head(2)

In [ ]:
dc = DataClean(df, configs)

In [ ]:
dc.select_cols()
dc.rename_cols()
dc.select_nnull_cols()
dc.select_nneg_cols()
dc.data_type()
df_tratada = dc.return_data()
df_tratada.head()

In [ ]:
df_tratada.shape

In [ ]:
for col in list(configs[configs["padroniza_str"] == 1]["nome"]):
    df_tratada[col] = df_tratada.loc[:,col].apply(lambda x: padroniza_str(x))

In [ ]:
df_tratada.head()

### Criação de novas colunas

In [ ]:
# Colocar aqui as novas colunas, importantando as funções desenvolvidas no trabalho

### Insere no banco de dados

In [ ]:
con = sqlite3.connect('projeto_python.db')
cur = con.cursor()
engine = create_engine("sqlite:///projeto_python.db")

In [ ]:
if not inspect(engine).has_table("flights"):
    create_db(engine)
    # Não esquecer de acrescentar as novas colunas na criação da base

In [ ]:
with engine.connect() as connection:
        df_tratada.to_sql(name='flights', con=connection, index=False, if_exists='append')

In [ ]:
cur.execute('SELECT * from flights limit 10')
result = cur.fetchall()
print(result)